___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4


#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange


1. Підключитися до API.

In [1]:
import requests
import datetime as d
import math
import collections
import pickle

URL = 'https://bank.gov.ua/NBU_Exchange/exchange_site'
CURRENCY = 'usd'
END_DATE = d.date.today()
PERIOD_IN_DAYS = 365

def date_format(date):
    return date.strftime('%Y%m%d')


params ={
    "start": date_format(END_DATE-d.timedelta(days=PERIOD_IN_DAYS)),
    "end": date_format(END_DATE), 
    "valcode": CURRENCY,
    "sort": 'exchangedate',
    "order": 'desc',
    "json": 'json',
    }


response = (requests.get(url=URL, params=params)).json()

2. Отримати курс долара за останній рік.

In [2]:
rates_list =[[d.datetime.strptime(day['exchangedate'],'%d.%m.%Y').strftime('%m-%Y'), day['rate']] for day in response]

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [3]:
rates_by_month = collections.defaultdict(list)

for month, rate in rates_list:
    rates_by_month[month].append(rate)

        
rates_analysis_by_mounth = []

for month, rates in rates_by_month.items():
    current_rates_analysis ={}
    # 1. Находим среднее арифметическое выборки.
    average_rate = sum(rates) / len(rates)
    # 2. От каждого значения выборки отнимаем среднее арифметическое. 3. Каждую полученную разницу возводим в квадрат.
    # 4. Суммируем полученные значения квадратов разниц. 5. Делим на размер выборки. 6. Находим квадратный корень.
    std_dev = math.sqrt(sum([(rate - average_rate)**2 for rate in rates]) / (len(rates)))
    current_rates_analysis['month'] = month
    current_rates_analysis['average_rate'] = round(average_rate, 4)
    current_rates_analysis['std_dev'] = round(std_dev, 4)
    rates_analysis_by_mounth.append(current_rates_analysis)




rates_analysis_by_mounth

[{'month': '03-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '02-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '01-2023', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '12-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '11-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '10-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '09-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '08-2022', 'average_rate': 36.5686, 'std_dev': 0.0},
 {'month': '07-2022', 'average_rate': 31.8501, 'std_dev': 3.4993},
 {'month': '06-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '05-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '04-2022', 'average_rate': 29.2549, 'std_dev': 0.0},
 {'month': '03-2022', 'average_rate': 29.2549, 'std_dev': 0.0}]

4. Дану інформацію записати у файл за допомогою pickle.

In [4]:
with open('rates_analysis.pickle', 'wb') as f:
    pickle.dump(rates_analysis_by_mounth, f)

with open('rates_analysis.pickle','rb') as f:
    rates_analysis = pickle.load(f)

print ('rates_analysis', rates_analysis)

rates_analysis [{'month': '03-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '02-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '01-2023', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '12-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '11-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '10-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '09-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '08-2022', 'average_rate': 36.5686, 'std_dev': 0.0}, {'month': '07-2022', 'average_rate': 31.8501, 'std_dev': 3.4993}, {'month': '06-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '05-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '04-2022', 'average_rate': 29.2549, 'std_dev': 0.0}, {'month': '03-2022', 'average_rate': 29.2549, 'std_dev': 0.0}]


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/products

In [ ]:
# from selenium import webdriver
# from bs4 import BeautifulSoup

# # устанавливаем путь к драйверу браузера
# # здесь используется Chrome, но можно использовать любой другой
# driver_path = '/путь/к/драйверу/chromedriver'

# # инициализируем экземпляр браузера
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # запуск браузера в фоновом режиме
# driver = webdriver.Chrome(driver_path, options=options)

# # проходим по первым 5 страницам магазина
# for i in range(1, 6):
#     url = f'https://example.com/shop?page={i}'
#     driver.get(url)

#     # ждем, пока страница полностью загрузится
#     driver.implicitly_wait(10)

#     # получаем HTML-код страницы
#     soup = BeautifulSoup(driver.page_source, 'html.parser')

#     # здесь выполняется ваш код парсинга страницы
#     # ...

# # закрываем браузер после выполнения цикла
# driver.quit()

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [5]:
import requests
from bs4 import BeautifulSoup

URL_AGROSEM = 'https://smallpacking.agrosem.ua/products'

headers = {
    'authority': 'www.google.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'x-client-data': 'CIu2yQEIorbJAQjBtskBCKmdygEIrMfKAQj4x8oBCKPNygEI3NXKAQin+csB',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'uk'
}


response_agrosem = requests.get(url=URL_AGROSEM, headers=headers)


soup = BeautifulSoup(response_agrosem.text, 'lxml')

In [6]:
items = soup.find_all('div', class_='item')
#article = items.find('div',class_='info')
items

[<div class="item" content='{"id":1542,"name":"\u0414\u041e\u0411\u0420\u0418\u0412\u041e \u0414\u041b\u042f \u0412\u0406\u0427\u041d\u041e\u0417\u0415\u041b\u0415\u041d\u0418\u0425 \u0414\u0415\u0420\u0415\u0412 \u0422\u0410 \u041a\u0423\u0429\u0406\u0412 \u041e\u0421\u0406\u041d\u042c 3 \u043a\u0433","cat":"NPK \u0434\u043b\u044f \u0491\u0440\u0443\u043d\u0442\u043e\u0432\u043e\u0433\u043e \u0432\u043d\u0435\u0441\u0435\u043d\u043d\u044f, \u0414\u043b\u044f \u0441\u0430\u0434\u0443 \u0456 \u0433\u043e\u0440\u043e\u0434\u0443","weight":"","url":"\/products\/1542","poster":"https:\/\/admin.agrosem.ua\/admin\/yii2images\/images\/image-by-item-and-alias?item=PdProduct1542&amp;dirtyAlias=dc55d01751-1.jpg","price":"799.00","prev":""}' data-id="1542">
 <div class="flags">
 <span class="sale">Акція</span>
 </div>
 <div class="rate"><span class="ico star" model-id="1542" model-type="product"></span></div>
 <div class="poster" style="--i-img:url(https://admin.agrosem.ua/admin/yii2images/images

In [7]:
# noinspection PyShadowingNames
def get_splited_item_info(item):
    item_info = item.find('div',class_='info').get_text().strip().split()
    return item_info


# noinspection PyShadowingNames
def get_item_name(info):
    item_name = ' '.join(info[:-2])
    return item_name


def get_item_weight_in_kg(info):
    info = info[-2:]
    if info[1] == 'гВага':
        weight = int(info[0])/1000
    else:
        weight = float(info[0])
    return weight


# noinspection PyShadowingNames
def get_item_price_in_uah(item):
    item_price = item.find('p',class_='price').get_text().split()
    item_price = float(item_price[0])
    return item_price


# noinspection PyShadowingNames
def get_article(item):
    article = item.get('data-id')
    return article


item_dict = {}

for item in soup.find_all('div', class_='item'):
    item_list = []
    article = get_article(item)
    item_info = get_splited_item_info(item)
    item_name = get_item_name(item_info)
    item_price = get_item_price_in_uah(item)
    item_weight_in_kg = get_item_weight_in_kg(get_splited_item_info(item))
    item_list.append(item_name)
    item_list.append(item_price)
    item_list.append(item_weight_in_kg)
    item_dict[article] = item_list

item_dict

{'1542': ['ДОБРИВО ДЛЯ ВІЧНОЗЕЛЕНИХ ДЕРЕВ ТА КУЩІВ ОСІНЬ', 799.0, 3.0],
 '1523': ['ДОБРИВО ДЛЯ ВІЧНОЗЕЛЕНИХ ДЕРЕВ ТА КУЩІВ ОСІНЬ', 2.0, 10.0],
 '1525': ['ДОБРИВО УНІВЕРСАЛЬНЕ ГРУНТОВЕ ОСІНЬ', 348.0, 1.0],
 '1528': ['ДОБРИВО ДЛЯ ГАЗОНУ ОСІНЬ', 348.0, 1.0],
 '1536': ['ДОБРИВО ДЛЯ ГАЗОНУ ОСІНЬ', 2.0, 10.0],
 '1608': ['ДОБРИВО ДЛЯ САМШИТУ ТА ВІЧНОЗЕЛЕНИХ ЧАГАРНИКІВ, Yara Folicare',
  119.0,
  0.18],
 '1610': ['ДОБРИВО ДЛЯ ХВОЙНИХ РОСЛИН, Yara Folicare', 169.0, 0.18],
 '1537': ['ДОБРИВО ДЛЯ ДЕКОРАТИВНИХ ЛИСТОВИХ ВЕСНА-ЛІТО', 799.0, 3.0],
 '1538': ['ДОБРИВО ДЛЯ ВІЧНОЗЕЛЕНИХ ДЕРЕВ ТА КУЩІВ ВЕСНА-ЛІТО', 2.0, 10.0],
 '1539': ['ДОБРИВО ДЛЯ ДЕКОРАТИВНИХ ЛИСТОВИХ ВЕСНА-ЛІТО', 348.0, 1.0],
 '1540': ['ДОБРИВО ДЛЯ ОВОЧІВ', 799.0, 3.0],
 '1541': ['ДОБРИВО ДЛЯ ВИНОГРАДУ', 348.0, 1.0]}

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

2. Виведіть перелік всіх таблиць.

3. Список всіх cust_id з таблиці Customers table.

4. Всю таблицю Customers table

5. Список клієнтів (cust_names) відсортованих від Z до A

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

8. Товар (з таблиці Products), ціна якого становить 9.49

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

10. Кількість товару, що було продано

11. Кількість найменувань товару, ціна якого більша за 4

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди

Краткая информация о базе данных "Компьютерная фирма":

Схема БД состоит из четырех таблиц:\
Product(maker, model, type)\
PC(code, model, speed, ram, hd, cd, price)\
Laptop(code, model, speed, ram, hd, price, screen)\
Printer(code, model, color, type, price)\
 - Таблица Product представляет производителя (maker), номер модели (model) и тип ('PC' - ПК, 'Laptop' - ПК-блокнот или 'Printer' - принтер). Предполагается, что номера моделей в таблице Product уникальны для всех производителей и типов продуктов.\
 - В таблице PC для каждого ПК, однозначно определяемого уникальным кодом – code, указаны модель – model (внешний ключ к таблице Product), скорость - speed (процессора в мегагерцах), объем памяти - ram (в мегабайтах), размер диска - hd (в гигабайтах), скорость считывающего устройства - cd (например, '4x') и цена - price (в долларах).\ 
 - Таблица Laptop аналогична таблице РС за исключением того, что вместо скорости CD содержит размер экрана -screen (в дюймах).\
 - В таблице Printer для каждой модели принтера указывается, является ли он цветным - color ('y', если цветной), тип принтера - type (лазерный – 'Laser', струйный – 'Jet' или матричный – 'Matrix') и цена - price.

Задание: 1 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd

In [ ]:
Select model,speed, hd
from PC
where price < 500

Задание: 2 (Serge I: 2002-09-21)
Найдите производителей принтеров. Вывести: maker

In [ ]:
Select distinct(maker)
from Product
where type = 'printer'

Задание: 3 (Serge I: 2002-09-30)
Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
Select model, ram, screen
from Laptop
where price > 1000

Задание: 4 (Serge I: 2002-09-21)
Найдите все записи таблицы Printer для цветных принтеров.

In [ ]:
Select *
from Printer
where color = 'y'

Задание: 5 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

In [ ]:
Select model, speed, hd
from PC
where cd in('12x','24x') and price < 600

Задание: 6 (Serge I: 2002-10-28)
Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.

In [ ]:
Select distinct p.maker, l.speed
from Product p
join Laptop l on l.model = p.model
where l.hd >= 10 and p.type = 'Laptop'
order by p.maker, l.speed

Задание: 7 (Serge I: 2002-11-02)
Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).

In [ ]:
Select l.model, l.price
from Product p
join Laptop l on l.model = p.model
where p.maker = 'B'
union
Select l.model, l.price
from Product p
join PC l on l.model = p.model
where p.maker = 'B'
union
Select l.model, l.price
from Product p
join Printer l on l.model = p.model
where p.maker = 'B'

Задание: 8 (Serge I: 2003-02-03)
Найдите производителя, выпускающего ПК, но не ПК-блокноты.

In [ ]:
Select distinct maker
from Product 
where type = 'PC' and maker not in (Select maker from Product where type = 'Laptop')

Задание: 9 (Serge I: 2002-11-02)
Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker

In [ ]:
Select distinct p.maker
from Product p
join PC pc on pc.model= p.model
where pc.speed >= 450

Задание: 10 (Serge I: 2002-09-23)
Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price

In [ ]:
Select model, price
from Printer 
where price = (select max(price) from Printer)

Задание: 11 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК.

In [ ]:
Select avg(speed)
from PC

Задание: 12 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
select avg(speed)
from Laptop
where price > 1000

Задание: 13 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК, выпущенных производителем A.

In [ ]:
Select avg(c.speed)
from PC c
join Product p on c.model = p.model
where p.maker = 'A'

Краткая информация о базе данных "Корабли":

Рассматривается БД кораблей, участвовавших во второй мировой войне. Имеются следующие отношения:\
Classes (class, type, country, numGuns, bore, displacement)\
Ships (name, class, launched)\
Battles (name, date)\
Outcomes (ship, battle, result)\
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.\
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах).\
В отношении Ships записаны название корабля, имя его класса и год спуска на воду.\
В отношение Battles включены название и дата битвы, в которой участвовали корабли.\
В отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).\
Замечания.
 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 
 2) Потопленный корабль в последующих битвах участия не принимает.

Задание: 14 (Serge I: 2002-11-05)
Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий.

In [ ]:
Select s.class, s.name, c.country
from Ships s
join Classes c on s.class = c.class
where c. numGuns >= 10

Задание: 15 (Serge I: 2003-02-03)
Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD

In [ ]:
Select hd
from PC
group by hd
having count(hd) > 1

Задание: 16 (Serge I: 2003-02-03)
Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

In [ ]:
Select distinct p.model, c.model, p.speed, p.ram
from PC p, PC c
where p.speed = c.speed and p.ram = c.ram and p.model > c.model
order by p.model desc, c.model desc

Задание: 17 (Serge I: 2003-02-03)
Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed

In [ ]:
select distinct p.type, l.model, l.speed
from Laptop l
join Product p on p.model = l.model
where l.speed < ALL(select speed from PC)

Задание: 18 (Serge I: 2003-02-03)
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price

In [ ]:
select distinct p1.maker, p2.price
from Product p1
join Printer p2 on p1.model = p2.model
where p2.color = 'y' and p2.price in (select min(price) from Printer where color = 'y')

Задание: 19 (Serge I: 2003-02-13)
Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов.
Вывести: maker, средний размер экрана.

In [ ]:
select p.maker, avg(l.screen)
from Product p
join Laptop l on p.model = l.model
where p.type = 'laptop'
group by p.maker

Задание: 20 (Serge I: 2003-02-13)
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

In [ ]:
select maker, count(distinct model) as num_models
from Product
where type = 'PC'
group by maker
having count(distinct model) >= 3

Задание: 21 (Serge I: 2003-02-13)
Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена.

In [ ]:
select p2.maker, max(p1.price)
from PC p1
join Product p2 on p1.model=p2.model
group by p2.maker

Задание: 22 (Serge I: 2003-02-13)
Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена.

In [ ]:
select speed, avg(price)
from PC
where speed > 600
group by speed

Задание: 23 (Serge I: 2003-02-14)
Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker

In [ ]:
select distinct maker 
from Product 
where maker IN (
  select maker 
  from Product p
  join PC on p.model = PC.model 
  where PC.speed >= 750
  intersect
  select maker 
  from Product p
  join Laptop l on p.model = l.model 
  where l.speed >= 750
)
and type in ('PC', 'Laptop')

Задание: 24 (Serge I: 2003-02-03)
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

In [ ]:
with top_price as (select model, price
from PC 
where price =
(select max(price) price
from PC)
union
select model, price
from Laptop 
where price =
(select max(price) price
from Laptop)
union
select model, price
from Printer 
where price =
(select max(price) price
from Printer))

select model from top_price
where price = (select max(price) from top_price)

Задание: 25 (Serge I: 2003-02-14)
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

In [ ]:
# Ваш запрос вернул правильные данные на основной базе, но не прошел тест на проверочной базе
select distinct maker
from Product
where type = 'Printer'
and maker in(
select maker
from Product
where model in(
select model
from PC
where speed in(
select max(speed)
from PC
where ram in(
select min(ram)
from PC))))

In [ ]:
# Ваш запрос не прошел проверку на третьей проверочной базе
with PC_Printer_maker as (select maker, ram, speed
from PC 
join Product p on p.model = PC.model
where maker in(
select distinct maker from Product
where type = 'Printer')
and type = 'PC')


select distinct maker from PC_Printer_maker
where ram in (select MIN(ram) from PC_Printer_maker)
and speed in (select max(speed) from PC_Printer_maker
where ram in (select MIN(ram) from PC_Printer_maker))

Задание: 26 (Serge I: 2003-02-14)
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

In [ ]:
select avg(price) as AVG_price
from (
select price
from PC
where model in(
select model
from Product
where maker = 'A' and type = 'PC')
union all
select price
from Laptop
where model in(
select model
from Product
where maker = 'A' and type = 'Laptop')
) as prices

Задание: 27 (Serge I: 2003-02-03)
Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD.

In [ ]:
select maker, avg(hd) as AVG_hd
from Product p 
join PC on p.model = PC.model
where maker in (
select maker
from Product
where type = 'Printer')
group by maker

Задание: 28 (Serge I: 2012-05-04)
Используя таблицу Product, определить количество производителей, выпускающих по одной модели.

In [ ]:
select count(one_model_maker)
from (
select count(model) one_model_maker
from Product
group by maker
having count(maker) = 1) as one_model_maker_table

Задание: 29 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.


Краткая информация о базе данных "Фирма вторсырья"

Фирма имеет несколько пунктов приема вторсырья. Каждый пункт получает деньги для их выдачи сдатчикам вторсырья. Сведения о получении денег на пунктах приема записываются в таблицу:\
Income_o(point, date, inc)\
Первичным ключом является (point, date). При этом в столбец date записывается только дата (без времени), т.е. прием денег (inc) на каждом пункте производится не чаще одного раза в день.\
Сведения о выдаче денег сдатчикам вторсырья записываются в таблицу:\
Outcome_o(point, date, out)
В этой таблице также первичный ключ (point, date) гарантирует отчетность каждого пункта о выданных деньгах (out) не чаще одного раза в день.\
В случае, когда приход и расход денег может фиксироваться несколько раз в день, используется другая схема с таблицами, имеющими первичный ключ code:\
Income(code, point, date, inc)\
Outcome(code, point, date, out)\
Здесь также значения столбца date не содержат времени.

In [ ]:
select i.*, o.out
from Income_o i
join Outcome_o o on i.point = o.point and i.date = o.date

Задание: 30 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).